# ModelRadar Tutorial Part 1 - Experiments

We need to run some experiments to apply modelradar. This notebook uses neuralforecast and statsforecast to train and test the following models: NHITS, KAN, MLP (2 hidden layers), and MLP (3 hidden layers).

Cross-validation is conducted based on Nixtla's framework.

- Loading the required libraries:

In [9]:
import warnings

import pandas as pd
from datasetsforecast.m3 import M3
from neuralforecast.models import NHITS, MLP, KAN
from neuralforecast import NeuralForecast
from statsforecast.models import SeasonalNaive
from statsforecast import StatsForecast

# !pip install modelradar -U
from modelradar.pipelines.data_splits import train_test_split

warnings.filterwarnings("ignore")

- Loading the dataset (m3 monthly) and splitting it into training and testing sets:

In [2]:
ds, *_ = M3.load('.', group='Monthly')

input_size, horizon = 12, 12

train_df, test_df = train_test_split(ds, horizon=horizon)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.76M/1.76M [00:00<00:00, 3.22MiB/s]
INFO:datasetsforecast.utils:Successfully downloaded M3C.xls, 1757696, bytes.
/Users/vcerq/miniconda/envs/modelradar/lib/python3.10/site-packages/datasetsforecast/m3.py:108: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(class_group.freq)


- Preparing the models based on neuralforecast and statsforecast and running cross-validation:

In [ ]:
models = [NHITS(h=horizon, input_size=input_size),
          KAN(h=horizon, input_size=input_size),
          MLP(h=horizon, input_size=input_size),
          MLP(h=horizon, input_size=input_size, num_layers=3)]

stats_models = [SeasonalNaive(season_length=12)]

nf = NeuralForecast(models=models, freq='ME')
sf = StatsForecast(models=stats_models, freq='ME', n_jobs=1)

cv_nf = nf.cross_validation(df=train_df, n_windows=2)
cv_sf = sf.cross_validation(df=train_df, h=horizon, level=[99])

cv = cv_nf.merge(cv_sf.drop(columns='y'), on=['unique_id', 'ds', 'cutoff'])

- Estimating anomalous observations based on a predictions interval approach using seasonal naive. Essentially, observations outside the 99%-confidence prediction interval of a seasonal naive are considered anomalous.

In [4]:
is_outside_pi = (cv['y'] >= cv['SeasonalNaive-hi-99']) | (cv['y'] <= cv['SeasonalNaive-lo-99'])
is_outside_pi = is_outside_pi.astype(int)
cv['is_anomaly'] = is_outside_pi.astype(int)

In [6]:
cv.head()

,ds,cutoff,NHITS,KAN,MLP,MLP1,y,SeasonalNaive,SeasonalNaive-lo-99,SeasonalNaive-hi-99,is_anomaly
unique_id,,,,,,,,,,,
M1,1993-09-30,1993-08-31,2522.375977,2832.563232,2227.387207,2108.703369,4800.0,6720.0,-1538.656675,14978.656675,0
M1,1993-10-31,1993-08-31,2222.809082,2208.655029,1891.918701,1820.084595,3000.0,2040.0,-6218.656675,10298.656675,0
M1,1993-11-30,1993-08-31,2850.925781,3215.884521,2641.873047,2418.422607,3120.0,6480.0,-1778.656675,14738.656675,0
M1,1993-12-31,1993-08-31,2324.294678,2065.045898,1888.080688,1995.671875,5880.0,1920.0,-6338.656675,10178.656675,0
M1,1994-01-31,1993-08-31,2614.612061,2493.655762,2245.066650,2192.622559,2640.0,3600.0,-4658.656675,11858.656675,0


- Storing cross-validation results in a csv file:

In [7]:
cv.to_csv('cv.csv')